# DistilBERT fine-tuning으로 감정 분석 모델 학습하기

이번 실습에서는 pre-trained된 DistilBERT를 불러와 이전 주차 실습에서 사용하던 감정 분석 문제에 적용합니다. 먼저 필요한 library들을 불러옵니다.

In [ ]:
!pip -q install tqdm boto3 requests regex sentencepiece sacremoses datasets

그 후, 우리가 사용하는 DistilBERT pre-training 때 사용한 tokenizer를 불러옵니다.

In [ ]:
import torch
from datasets import load_dataset
from torch.utils.data import DataLoader

tokenizer = torch.hub.load('huggingface/pytorch-transformers', 'tokenizer', 'distilbert-base-uncased')

DistilBERT의 tokenizer를 불러왔으면 이제 `collate_fn`과 data loader를 정의합니다. 이 과정은 이전 실습과 동일하게 다음과 같이 구현할 수 있습니다.

In [19]:
ds = load_dataset("fancyzhx/ag_news")
# ds = load_dataset("stanfordnlp/imdb")
from torch.utils.data import Subset

def collate_fn(batch):
  texts = [row['text'] for row in batch]
  labels = [row['label'] for row in batch]

  encoded = tokenizer(texts, padding=True, truncation=False, return_tensors='pt')
  texts = encoded['input_ids']
  masks = encoded['attention_mask']
  labels = torch.LongTensor(labels)

  return texts, masks, labels


ds = load_dataset("fancyzhx/ag_news")
# BATCH_SIZE = len(ds['train'])
BATCH_SIZE = 32
train_loader = DataLoader(
    ds['train'], batch_size=BATCH_SIZE, shuffle=True, collate_fn=collate_fn
)
test_loader = DataLoader(
    ds['test'], batch_size=BATCH_SIZE, shuffle=False, collate_fn=collate_fn
)
# train_loader = DataLoader(
#     Subset(ds['train'],range(0,32)), batch_size=BATCH_SIZE, shuffle=True, collate_fn=collate_fn
# )
# test_loader = DataLoader(
#     Subset(ds['test'], range(0,32)), batch_size=BATCH_SIZE, shuffle=False, collate_fn=collate_fn
# )

이제 pre-trained DistilBERT를 불러옵니다. 이번에는 PyTorch hub에서 제공하는 DistilBERT를 불러봅시다.

In [ ]:
model = torch.hub.load('huggingface/pytorch-transformers', 'model', 'distilbert-base-uncased')
model

출력 결과를 통해 우리는 DistilBERT의 architecture는 일반적인 Transformer와 동일한 것을 알 수 있습니다.
Embedding layer로 시작해서 여러 layer의 Attention, FFN를 거칩니다.

이제 DistilBERT를 거치고 난 `[CLS]` token의 representation을 가지고 text 분류를 하는 모델을 구현합시다.

In [20]:
from torch import nn

class TextClassifier(nn.Module):
  def __init__(self):
    super().__init__()
    self.encoder = torch.hub.load('huggingface/pytorch-transformers', 'model', 'distilbert-base-uncased')
    self.classifier = nn.Linear(self.encoder.config.dim, 4)
    self.dropout = nn.Dropout(self.encoder.config.seq_classif_dropout)
    for param in self.encoder.parameters():
        param.requires_grad = False
      
  def forward(self, input_ids, attention_mask=None):
    # print(attention_mask)
    x = self.encoder(input_ids=input_ids, attention_mask=attention_mask)['last_hidden_state']
    # cls_embedding = outputs.last_hidden_state[:, 0, :]  # Get the [CLS] token's output (batch_size, hidden_size)
    # x = self.dropout(outputs)
    logits = self.classifier(x[:,0])
    return logits
        
model = TextClassifier()

Using cache found in /Users/heekyung/.cache/torch/hub/huggingface_pytorch-transformers_main


위와 같이 `TextClassifier`의 `encoder`를 불러온 DistilBERT, 그리고 `classifier`를 linear layer로 설정합니다.
그리고 `forward` 함수에서 순차적으로 사용하여 예측 결과를 반환합니다.

다음은 마지막 classifier layer를 제외한 나머지 부분을 freeze하는 코드를 구현합니다.

In [ ]:
for param in model.encoder.parameters():
  param.requires_grad = False

위의 코드는 `encoder`에 해당하는 parameter들의 `requires_grad`를 `False`로 설정하는 모습입니다.
`requires_grad`를 `False`로 두는 경우, gradient 계산 및 업데이트가 이루어지지 않아 결과적으로 학습이 되지 않습니다.
즉, 마지막 `classifier`에 해당하는 linear layer만 학습이 이루어집니다.
이런 식으로 특정 부분들을 freeze하게 되면 효율적으로 학습을 할 수 있습니다.

마지막으로 이전과 같은 코드를 사용하여 학습 결과를 확인해봅시다.

In [22]:
import itertools
from datetime import datetime
from torch.optim import Adam

loss_fn = nn.CrossEntropyLoss()
lr = 0.001
# loss_fn = nn.BCEWithLogitsLoss()
device = 'cuda' if torch.cuda.is_available() else 'mps'
model = model.to(device)
optimizer = Adam(model.parameters(), lr=lr)
# lr = 3e-5
n_epochs = 5

for epoch in range(n_epochs):
  total_loss = 0.
  model.train()
  for (inputs, masks, labels) in itertools.islice(train_loader, 100):
    model.zero_grad()
    inputs, masks, labels = inputs.to(device), masks.to(device), labels.to(device).float()
    preds = model(inputs, masks)
    # preds = model(inputs, masks)[..., 0] # loss over 70,000
    loss = loss_fn(preds, labels)
    loss.backward()
    optimizer.step()
    total_loss += loss.item()

  print(f"Epoch {epoch:3d} | Train Loss: {total_loss}")

Epoch   0 | Train Loss: 78.12759348750114
Epoch   1 | Train Loss: 43.71672946214676
Epoch   2 | Train Loss: 40.57450167834759
Epoch   3 | Train Loss: 38.12422529608011
Epoch   4 | Train Loss: 36.38934251666069


In [25]:
def accuracy(model, dataloader):
  cnt = 0
  acc = 0

  for (inputs, masks, labels) in dataloader:
    inputs, masks, labels = inputs.to(device), masks.to(device), labels.to(device)
    preds = model(inputs, 
                  
                  
                  masks)
    preds = torch.argmax(preds, dim=-1)
    # preds = (preds > 0).long()[..., 0]

    cnt += labels.shape[0]
    acc += (labels == preds).sum().item()

  return acc / cnt


with torch.no_grad():
  model.eval()
  train_acc = accuracy(model, train_loader)
  test_acc = accuracy(model, test_loader)
  print(f"=========> Train acc: {train_acc:.3f} | Test acc: {test_acc:.3f}")

=========> Train acc: 0.886 | Test acc: 0.888


Loss가 잘 떨어지고, 이전에 우리가 구현한 Transformer보다 더 빨리 수렴하는 것을 알 수 있습니다.

In [ ]:
# ds = load_dataset("fancyzhx/ag_news")

# def collate_fn(batch):
#   texts = [rows['text'] for rows in batch]
#   texts = torch.LongTensor(tokenizer(texts, padding=True, truncation=False ).input_ids)
#   return texts, labels

# train_loader = DataLoader(
#     ds['train'], batch_size=len(ds['train']), shuffle=True, collate_fn=collate_fn
# )

# for idx, (inputs, labels) in enumerate(train_loader):
#     print(idx, inputs.shape)